# Overall playground to see how to connect AutoDraft to labelme

## First we need to dive into the JSON file that labelme creates in order to understand how to create one that it can then operate on

In [1]:
# imports
import numpy as np
import json
import pandas as pd
import base64
import io
from pprint import pprint
import PIL.Image
from labelme import utils
import os.path as osp
from labelme import PY2
from labelme import QT4
import base64
from labelme import __version__
#from IPython.core.debugger import set_trace

#auto-reload of python modules
%load_ext autoreload
%autoreload 2

### First lets figure out how to use modules

As long as the module is located within the same notebook directory it is simple to import directly. Ensure to import without the .py ending otherwise it will not import as a module.

In [2]:
import my_module

I am being executed!


In [3]:
my_module.square(4)

16

In [4]:
help(my_module)

Help on module my_module:

NAME
    my_module

DESCRIPTION
    Our first Python module. This initial string is a module-level documentation string.
    It is not a necessary component of the module. It is a useful way to describe the
    purpose of your module.

FUNCTIONS
    cube(x)
    
    square(x)

DATA
    some_list = ['a', 1, None]

FILE
    /Users/blank/AutoDraft/imagery/labelme_playground/my_module.py




### Viewing JSON data
JSON data when loaded into python prints extra ugly. The following allows for much more readable viewing of JSON files

In [5]:
data = json.load(open("ship1.json"))
# print(data) ## the data prints in a difficult string fashion
# pprint(data) ## one very quick way to print out the data. Looks okay...

In [6]:
pretty_data = json.dumps(data, ensure_ascii=False, indent=4) # prints JSON beautifully with indentations
#print(pretty_data)

### Now we need to figure out how to view/save image data

First method will be to attmept to use code within labelme

In [7]:
# Function provided in labelme source code
def load_image_file(filename):
    try:
        image_pil = PIL.Image.open(filename)
    except IOError:
        logger.error("Failed opening image file: {}".format(filename))
        return

    # apply orientation to image according to exif
    image_pil = utils.apply_exif_orientation(image_pil)

    with io.BytesIO() as f:
        ext = osp.splitext(filename)[1].lower()
        if PY2 and QT4:
            format = "PNG"
        elif ext in [".jpg", ".jpeg"]:
            format = "JPEG"
        else:
            format = "PNG"
        image_pil.save(f, format=format)
        f.seek(0)
        return f.read()

Now we can see how this actually works....

In [8]:
data = load_image_file("ship1.jpg")
#print(data)

Now we convert image data to base64, and then decode it to utf-8 for reading

In [9]:
new_data = base64.b64encode(data).decode("utf-8")
#print(new_data)

Great success!!! The above matches the data found in the applicable json file.
## Writing JSON files
Now we need to figure out how to write a simple JSON file, and then create one from scratch that labelme will open...the goal is just to write one that will match the current JSON file and that labelme will then open it successfully!

In [10]:
class LabelFile:
    
    #initializes the class
    def __init__(self, filename=None, shapes=[]):
        self.shapes = shapes
        self.image_path = None
        self.image_data = None
        self.load(filename)
        self.filename = filename

    
    def load(self, filename):
        image_data = load_image_file(filename)
        image_path = osp.dirname(filename)+filename #this causes the directory of the image file plus the filename to be stored
        flags = {} #this portion would need updating based on flags we want to add
        shapes = [
                dict(
                    label="waterline", # we would fill this with the different label strings
                    points=[[209,77],[228,81],[228,99],[210,99],[208,88]], # x-y point of shape that is being drawn
                    shape_type="polygon", #we can dictate more shapes here but this seems to be the most robust
                    flags={}, #flags that we create
                    group_id=None, # used for gouping similar items
                    other_data=None, # not really sure.....
                ) 
                #for s in data["shapes"]
            ]
        self.shapes = shapes
        self.image_path = image_path
        self.image_data = image_data
        self.flags = flags
        
        
    
    #this part checks the height and width of image and stores it
    @staticmethod
    def _check_image_height_and_width(image_data):
        img_arr = utils.img_b64_to_arr(image_data)
        image_height = img_arr.shape[0]
        image_width = img_arr.shape[1]
        return image_height, image_width
    
    #this portion saves and writes the JSON file
    def save(
        self,
        filename,
        shapes,
        image_path,
        image_data,
        image_height=None,
        image_width=None,
        other_data=None,
        flags=None,
    ):
        if image_data is not None:
            image_data = base64.b64encode(image_data).decode("utf-8")
            image_height, image_width = self._check_image_height_and_width(image_data)
        if other_data is None:
            other_data = {}
        if flags is None:
            flags = {}
        data = dict(
            version=__version__,
            flags=flags,
            shapes=shapes,
            image_path=image_path,
            image_data=image_data,
            image_height=image_height,
            image_width=image_width,
        )
        try:
            if filename.endswith(".jpg"):
                filename = filename[:-4]
            with open(filename+".json", "w") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            #self.filename = filename
        except Exception as e:
            raise LabelFileError(e)
        

Alright now time to see what this produces

In [11]:
init = LabelFile("ship1_copy.jpg")

In [12]:
init.save(init.filename, init.shapes, init.image_path, init.image_data)

## Hell yeah this totally worked!!!

In [13]:
data = json.load(open("ship1_copy.json"))
pretty_data = json.dumps(data, ensure_ascii=False, indent=4) # prints JSON beautifully with indentations
#print(pretty_data)

## Iterating through an array

In [14]:
boxes = np.array([[219,182,2,17],
 [232,173,8,12],
 [232,153,7,11],
 [231,132,6,10],
 [230,111,7,12],
 [230,92,9,9],
 [216,79,13,22],
 [208,78,6,21],
 [229,72,7,11],
 [229,53,7,10]])

Okay, so what exactly do I want the code to do?  
- create new array
- array should look like $[x,\ y,\ x + w,\ y + h]$. From the original boxes array this should look like [boxes[0,0], boxes[0,1], boxes[0,0] + boxes[0,2], boxes[0,1] + boxes[0,3]]
- Iterate across each row would probable be the easiest method
- Also want to turn this into a dictionary that can be easily used by the next section of the labelme code

Pseudo code should look like:

    for i in boxes:
        go through each i row:
            do the operations above and assign them to empty array
    once new array is created make dictionary that matches labelme shapes dictionary
    shapes = [
                dict(
                    label="waterline", # we would fill this with the different label strings
                    points=[[209,77],[228,81],[228,99],[210,99],[208,88]], # x-y point of shape that is being drawn
                    shape_type="polygon", #we can dictate more shapes here but this seems to be the most robust
                    flags={}, #flags that we create
                    group_id=None, # used for gouping similar items
                    other_data=None, # not really sure.....
                ) 
                #for s in data["shapes"]
            ]
    return shapes dictionary
    

In [15]:
boxes.shape

(10, 4)

In [16]:
print(boxes[1,0])

232


Need to create a new JSON encoder to handle numpy arrays

In [17]:
class myEncoder(json.JSONEncoder):
    # Handles the default behavior of
    # the encoder when it parses an object 'obj'
    def default(self, obj):
        # If the object is a numpy array
        if isinstance(obj, np.ndarray):
            # Convert to Python List
            return obj.tolist()
        else:
            # Let the base class Encoder handle the object
            return json.JSONEncoder.default(self, obj)
 

**Should try and make this universal for boxes, lines, etc**

In [18]:
def extract_points(boxes):
    points = np.empty(boxes.shape)
    for i in range(0,10):
        points[i] = [boxes[i,0], boxes[i,1], boxes[i,0] + boxes[i,2], boxes[i,1] + boxes[i,3]]
       
    # Now create shapes dictionary for transfer to labelme
    shapes = [
        dict(
            label="draftmarks",
            points=points, # x-y point of shape that is being drawn
            shape_type="polygon", #we can dictate more shapes here but this seems to be the most robust
            flags={}, #flags that we create
            group_id=None, # used for gouping similar items
            other_data=None, # not really sure.....
        )
    ]
    return points, shapes

In [19]:
points, shapes = extract_points(boxes)

In [20]:
print(shapes)

[{'label': 'draftmarks', 'points': array([[219., 182., 221., 199.],
       [232., 173., 240., 185.],
       [232., 153., 239., 164.],
       [231., 132., 237., 142.],
       [230., 111., 237., 123.],
       [230.,  92., 239., 101.],
       [216.,  79., 229., 101.],
       [208.,  78., 214.,  99.],
       [229.,  72., 236.,  83.],
       [229.,  53., 236.,  63.]]), 'shape_type': 'polygon', 'flags': {}, 'group_id': None, 'other_data': None}]


In [21]:
with open("shapes"+".json", "w") as f:
    json.dump(shapes, f, ensure_ascii=False, indent=2, cls=myEncoder)

In [22]:
pretty_shapes = json.dumps(shapes, ensure_ascii=False, indent=4, cls=myEncoder)
#print(pretty_shapes)

## Okay so now time to see how to create the class file we need for our particular code
We'll need to cut out a few things, like the shapes parameter so that we can insert that from our code directly, and then re-write the load function to better suit our purposes
Also need to consider what other information needs to come from our code directly

In [34]:
class LabelFile:
    
    #initializes the class
    def __init__(self, filename=None, shapes=[]):
        self.shapes = shapes
        self.imagePath = None
        self.imageData = None
        self.load(filename)
        self.filename = filename

    
    def load(self, filename):
        image_data = load_image_file(filename)
        image_path = osp.dirname(filename)+filename #this causes the directory of the image file plus the filename to be stored
        flags = {} #this portion would need updating based on flags we want to add
        self.imagePath = image_path
        self.imageData = image_data
        self.flags = flags
        
        
    
    #this part checks the height and width of image and stores it
    @staticmethod
    def _check_image_height_and_width(imageData):
        img_arr = utils.img_b64_to_arr(imageData)
        image_height = img_arr.shape[0]
        image_width = img_arr.shape[1]
        return image_height, image_width
    
    #this portion saves and writes the JSON file
    def save(
        self,
        filename,
        shapes,
        image_path,
        image_data,
        image_height=None,
        image_width=None,
        other_data=None,
        flags=None,
    ):
        if image_data is not None:
            image_data = base64.b64encode(image_data).decode("utf-8")
            image_height, image_width = self._check_image_height_and_width(image_data)
        if other_data is None:
            other_data = {}
        if flags is None:
            flags = {}
        data = dict(
            version=__version__,
            flags=flags,
            shapes=shapes,
            imagePath=image_path,
            imageData=image_data,
            imageHeight=image_height,
            imageWidth=image_width,
        )
        if filename.endswith(".jpg"):
            filename = filename[:-4]
        with open(filename+".json", "w") as f:
            json.dump(data, f, ensure_ascii=False, indent=2, cls=myEncoder)

In [35]:
init = LabelFile("frame_0_meters_001.jpg", shapes)

In [37]:
init.save(init.filename, init.shapes, init.imagePath, init.imageData)

### Need to solve the issue of having multiple dictionary entries per shape

In [38]:
shapes1 = [
    dict(
        label="waterline", # we would fill this with the different label strings
        points=[[209,77],[228,81],[228,99],[210,99],[208,88]], # x-y point of shape that is being drawn
        shape_type="polygon", #we can dictate more shapes here but this seems to be the most robust
        flags={}, #flags that we create
        group_id=None, # used for gouping similar items
        other_data=None, # not really sure.....
    )
]

In [39]:
shapes2 = dict(
        label="waterline", # we would fill this with the different label strings
        points=[[209,77],[228,81],[228,99],[210,99],[208,88]], # x-y point of shape that is being drawn
        shape_type="polygon", #we can dictate more shapes here but this seems to be the most robust
        flags={}, #flags that we create
        group_id=None, # used for gouping similar items
        other_data=None, # not really sure.....
    )

In [40]:
shapes1.append(shapes2)

In [41]:
print(shapes1)

[{'label': 'waterline', 'points': [[209, 77], [228, 81], [228, 99], [210, 99], [208, 88]], 'shape_type': 'polygon', 'flags': {}, 'group_id': None, 'other_data': None}, {'label': 'waterline', 'points': [[209, 77], [228, 81], [228, 99], [210, 99], [208, 88]], 'shape_type': 'polygon', 'flags': {}, 'group_id': None, 'other_data': None}]


In [43]:
pretty_shapes1 = json.dumps(shapes1, ensure_ascii=False, indent=4, cls=myEncoder)
print(pretty_shapes1)

[
    {
        "label": "waterline",
        "points": [
            [
                209,
                77
            ],
            [
                228,
                81
            ],
            [
                228,
                99
            ],
            [
                210,
                99
            ],
            [
                208,
                88
            ]
        ],
        "shape_type": "polygon",
        "flags": {},
        "group_id": null,
        "other_data": null
    },
    {
        "label": "waterline",
        "points": [
            [
                209,
                77
            ],
            [
                228,
                81
            ],
            [
                228,
                99
            ],
            [
                210,
                99
            ],
            [
                208,
                88
            ]
        ],
        "shape_type": "polygon",
        "flags": {},
        

Alright that appears to have worked! So use .append to append whole dictionaries each loop

In [44]:
print(boxes)

[[219 182   2  17]
 [232 173   8  12]
 [232 153   7  11]
 [231 132   6  10]
 [230 111   7  12]
 [230  92   9   9]
 [216  79  13  22]
 [208  78   6  21]
 [229  72   7  11]
 [229  53   7  10]]


In [99]:
def transform_points(points):
    column = points.shape[0]
    transformed_points = np.empty(((column//2), 2))
    n = 1
    i = 1
    transformed_points[0] = [points[0], points[1]]
    while i <= 7:
        transformed_points[n] = [points[i+1], points[i+2]]
        if i+2 == column - 1:
            break
        i+=2
        n+=1
    
    return transformed_points

In [101]:
love = transform_points(points[1])
print(love)

[[232. 173.]
 [232. 185.]
 [240. 173.]
 [240. 185.]]


In [111]:
def extract_points(boxes):
    # rearrange to get proper sizing
    row, column = boxes.shape
    points = np.empty((row, column*2))
    for i in range(0,row):
        points[i] = [boxes[i,0], boxes[i,1],
                     boxes[i,0], boxes[i,1] + boxes[i,3],
                     boxes[i,0] + boxes[i,2], boxes[i,1],
                     boxes[i,0] + boxes[i,2], boxes[i,1] + boxes[i,3]]
    
    
       
    # Now create shapes dictionary for transfer to labelme
    shapes = []
    for i in range(0,row):
        transformed_points = transform_points(points[i])
        shapes.append(
            dict(
                label="draftmarks",
                points=transformed_points, # x-y point of shape that is being drawn
                shape_type="polygon", #we can dictate more shapes here but this seems to be the most robust
                flags={}, #flags that we create
                group_id=None, # used for gouping similar items
                other_data=None, # not really sure.....
            )
        )
    return points, shapes

In [60]:
test = np.empty(boxes.shape)
size = boxes.shape
print(size)
row, column = size
print(row)
print(column)
column = column*2
size = (row,column)
test = np.empty(size)
test.shape

(10, 4)
10
4


(10, 8)

In [112]:
points, shapes = extract_points(boxes)
points.shape
points.shape[0]
points.shape[1]
row, column = points.shape
print(row, column)
love = points[1]
love.shape
8//2

10 8


4

In [113]:
pretty_shapes = json.dumps(shapes, ensure_ascii=False, indent=4, cls=myEncoder)
print(pretty_shapes)

[
    {
        "label": "draftmarks",
        "points": [
            [
                219.0,
                182.0
            ],
            [
                219.0,
                199.0
            ],
            [
                221.0,
                182.0
            ],
            [
                221.0,
                199.0
            ]
        ],
        "shape_type": "polygon",
        "flags": {},
        "group_id": null,
        "other_data": null
    },
    {
        "label": "draftmarks",
        "points": [
            [
                232.0,
                173.0
            ],
            [
                232.0,
                185.0
            ],
            [
                240.0,
                173.0
            ],
            [
                240.0,
                185.0
            ]
        ],
        "shape_type": "polygon",
        "flags": {},
        "group_id": null,
        "other_data": null
    },
    {
        "label": "draftmarks",
       

In [114]:
with open("shapes"+".json", "w") as f:
    json.dump(shapes, f, ensure_ascii=False, indent=2, cls=myEncoder)

Woohoo!!! This is finally in the correct shape!!!